# Cora Dataset 
Description of the dataset ....

In [1]:
import os
import time
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import distanceclosure as dc
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx, from_networkx
from torch.nn import Linear
from torch_geometric.nn import GCNConv
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from src import add_weights_to_edges, mb_sparsify, generate_train_and_test_mask, GCN, train, test, eval_model

ModuleNotFoundError: No module named 'train'

In [2]:


dataset = Planetoid(root='../data', name='cora', transform=T.NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Has isolated nodes: False
Has self-loops: False
Is undirected: True


## Preprocessing the data
We separate the experiment into three categories of data
- The original graph
- The metric backbone of the graph with edges weighted by the inverse of their adamic-adar index
- The metric backbone of the graph with edges weighted by the inverse of their jaccard coefficient

In [3]:

G = to_networkx(data, to_undirected=True, node_attrs=data.node_attrs())
J = add_weights_to_edges(G, method='jaccard')
AA = add_weights_to_edges(G, method='adamic_adar')

#Using rust to compute the metric backbone
AA_mb = nx.from_numpy_array(mb_sparsify(nx.to_numpy_array(AA)))
J_mb = nx.from_numpy_array(mb_sparsify(nx.to_numpy_array(J)))

print(f'Original Graph : {data.num_edges} edges')
print(f'Metric Backbone of Adamic Adar weighted graph : {AA_mb.number_of_edges()} edges -> {AA_mb.number_of_edges()/data.num_edges*100:.2f}% of the edges remain')
print(f'Metric Backbone of Jaccard weighted graph : {J_mb.number_of_edges()} edges -> {J_mb.number_of_edges()/data.num_edges*100:.2f}% of the edges remain')

data_AdamicAdar_mb = from_networkx(AA_mb)
data_Jaccard_mb = from_networkx(J_mb)
dataG = data

Original Graph : 10556 edges
Metric Backbone of Adamic Adar weighted graph : 4653 edges -> 44.08% of the edges remain
Metric Backbone of Jaccard weighted graph : 4858 edges -> 46.02% of the edges remain


In [5]:
train_mask, test_mask = generate_train_and_test_mask(data.num_nodes)
G_losses = []
G_accuracy = []
EPOCHS = range(100)

fg_model = GCN(dataG, hidden_channels=16, num_classes=dataset.num_classes)
fg_criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
fg_optimizer = torch.optim.Adam(fg_model.parameters(), lr=0.01)  # Define optimizer.
eval_model()

plt.legend()
plt.show()

# print(f'{G_losses[0]} -> {G_losses[-1]}')
# print(f'{AA_losses[0]} -> {AA_losses[-1]}')
og_test_acc = test(dataG)

TypeError: GCN.forward() takes 3 positional arguments but 4 were given